<a href="https://colab.research.google.com/github/goyal26sachin/Spoon-Knife/blob/master/Momentum_Strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os, sys, time, math
from pathlib import Path
import pandas as pd
import numpy as np

# optional - install yfinance when running in Colab or new env
try:
    import yfinance as yf
except Exception:
    print("yfinance not installed. Install with: pip install yfinance")
    # raise or continue; user should install

# -------------------------
# CONFIG / Defaults
# -------------------------
DEFAULT_INPUT_TICKER_XLSX = "/content/Input_List_of_Stocks.xlsx"     # <---- "/mnt/data/BSE-500.xlsx"   # <--- change to /content/BSE-500.xlsx in Colab if needed
OUTPUT_DIR = Path("./output")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# -------------------------
# Helper utilities
# -------------------------
def confirm(prompt="Proceed? (y/n): "):
    ans = input(prompt).strip().lower()
    return ans in ("y","yes")

def safe_save_df_to_excel(writer, df, sheet_name):
    # Excel sheet name limit 31 chars
    name = str(sheet_name)[:31]
    try:
        df.to_excel(writer, sheet_name=name, index=False)
    except Exception as e:
        # fallback: truncate further
        df.to_excel(writer, sheet_name=name[:25], index=False)

# -------------------------
# Module 1: Data downloader
# -------------------------
def download_yahoo_data(ticker_list_file=DEFAULT_INPUT_TICKER_XLSX,
                        output_path=OUTPUT_DIR / "yahoo_data.xlsx",
                        verbose=True,
                        append_suffix=".NS",
                        years=3): # Added years parameter with default 3
    """
    Reads tickers from input XLSX (first sheet, first column or 'Ticker'/'Symbol' col).
    Downloads OHLCV + Adj Close and basic info (trailingPE/forwardPE) for each ticker via yfinance.
    Saves per-symbol sheet + Combined_Data + PE_Data + Errors to output_path.
    Returns path to saved file.
    """
    print("Reading ticker list from:", ticker_list_file)
    xlf = pd.ExcelFile(ticker_list_file)
    sheet = xlf.sheet_names[0]
    df_list = pd.read_excel(ticker_list_file, sheet_name=sheet)
    # detect ticker column
    ticker_col = None
    for c in df_list.columns:
        if str(c).strip().lower() in ("ticker","symbol","code"):
            ticker_col = c; break
    if ticker_col is None:
        ticker_col = df_list.columns[0]
    raw_tickers = df_list[ticker_col].astype(str).str.strip().dropna().unique().tolist()
    # Automatically append .NS if not present
    tickers = [t if "." in t else t + append_suffix for t in raw_tickers if t and str(t).upper() not in ("NAN","NONE","")]
    print(f"Found {len(tickers)} tickers (sample):", tickers[:10])
    sheets = {}
    pe_records = []
    errors = []
    for i, sym in enumerate(tickers, start=1):
        try:
            print(f"[{i}/{len(tickers)}] Downloading {sym} ...", end=" ")
            ticker = yf.Ticker(sym)
            hist = ticker.history(period=f"{years}y", auto_adjust=False) # Used years parameter
            if hist is None or hist.shape[0]==0:
                hist = ticker.history(period=f"{years}y", auto_adjust=False) # Used years parameter
            if hist is None or hist.shape[0]==0:
                print("NO DATA")
                errors.append((sym,"no_history"))
                continue
            df_hist = hist.reset_index().rename(columns={'Date':'Date'})

            # Prioritize 'Adj Close', if not available use 'Close'
            price_col = 'Adj Close' if 'Adj Close' in df_hist.columns else ('Close' if 'Close' in df_hist.columns else None)

            if price_col is None:
                print("No price data found (Adj Close or Close)")
                errors.append((sym, "no_price_data"))
                continue

            df_hist = df_hist[['Date', price_col, 'Volume']].copy() # Added 'Volume' here
            df_hist.rename(columns={price_col: 'Adj Close'}, inplace=True) # Standardize column name to 'Adj Close'

            # Convert 'Date' column to be timezone-naive if it's timezone-aware
            if 'Date' in df_hist.columns and isinstance(df_hist['Date'].dtype, pd.DatetimeTZDtype):
                df_hist['Date'] = df_hist['Date'].dt.tz_localize(None)

            df_hist["Symbol"] = sym
            sheets[sym] = df_hist
            info = ticker.info if hasattr(ticker,'info') else {}
            pe_records.append({"Symbol": sym, "TrailingPE": info.get("trailingPE", None), "ForwardPE": info.get("forwardPE", None)})
            print(f"done ({df_hist.shape[0]} rows)")
        except Exception as e:
            print("error:", e)
            errors.append((sym,str(e)))
    # save to excel
    output_path = Path(output_path)
    with pd.ExcelWriter(output_path, engine="openpyxl", datetime_format="YYYY-MM-DD") as writer:
        for sym, df in sheets.items():
            safe_save_df_to_excel(writer, df, sym[:31])
        if len(sheets)>0:
            combined = pd.concat(sheets.values(), ignore_index=True, sort=False)
            safe_save_df_to_excel(writer, combined, "Combined_Data")
        safe_save_df_to_excel(writer, pd.DataFrame(pe_records), "PE_Data")
        safe_save_df_to_excel(writer, pd.DataFrame(errors, columns=["Symbol","Error"]), "Errors")
    print("Saved downloader output to:", output_path)
    return output_path

# -------------------------
# Module 2: Data cleanup
# -------------------------
def clean_data(yahoo_xlsx_path, output_clean_path=OUTPUT_DIR/"cleaned_price_matrix.xlsx", min_history_days=260, pct_missing_threshold=0.2):
    """
    Loads the 'Combined_Data' sheet or builds a wide price matrix from per-symbol sheets.
    Outputs:
       - Price matrix (wide)
       - Per-symbol summary with pct missing, zeros, spikes, stale runs, first/last valid date
    Also saves cleaned price matrix with only valid symbols (>= min_history_days non-null)
    Returns (clean_price_df, summary_df)
    """
    print("Loading yahoo data file:", yahoo_xlsx_path)
    xl = pd.ExcelFile(yahoo_xlsx_path)
    if "Combined_Data" in xl.sheet_names:
        df = pd.read_excel(yahoo_xlsx_path, sheet_name="Combined_Data", parse_dates=["Date"])
    else:
        # reconstruct combined from first sheet's symbol sheets
        # load every sheet and append
        frames = []
        for s in xl.sheet_names:
            if s in ("Combined_Data","PE_Data","Errors"): continue
            tmp = pd.read_excel(yahoo_xlsx_path, sheet_name=s, parse_dates=["Date"])
            if "Symbol" not in tmp.columns:
                tmp["Symbol"] = s
            frames.append(tmp)
        df = pd.concat(frames, ignore_index=True)
    # pivot to price matrix wide using Adj Close (preferred) else Close
    df['Adj Close'] = df.get('Adj Close', df.get('AdjClose', np.nan))
    if df['Adj Close'].notna().sum() > 0:
        price_series = df.pivot(index="Date", columns="Symbol", values="Adj Close")
    else:
        price_series = df.pivot(index="Date", columns="Symbol", values="Close")
    price_series = price_series.sort_index()
    # compute per-symbol metrics
    report = []
    total_days = price_series.shape[0]
    for sym in price_series.columns:
        ser = pd.to_numeric(price_series[sym], errors='coerce')
        non_null = ser.notna().sum()
        pct_missing = 1 - (non_null / total_days) if total_days>0 else 1.0
        zeros = (ser == 0).sum()
        negatives = (ser < 0).sum()
        unique_vals = ser.nunique(dropna=True)
        first_valid = ser.first_valid_index()
        last_valid = ser.last_valid_index()
        min_p = ser.min(skipna=True)
        max_p = ser.max(skipna=True)
        pct = ser.pct_change(fill_method=None).replace([np.inf,-np.inf], np.nan) # Added fill_method=None
        max_abs = float(np.nanmax(np.abs(pct.values))) if pct.notna().any() else np.nan
        days_over_50 = int((np.abs(pct) > 0.5).sum())
        # stale runs
        longest_stale = 0
        prev = None
        run = 0
        for v in ser.values:
            if pd.isna(v):
                prev = None; run = 0; continue
            if prev is None or v != prev:
                run = 1; prev = v
            else:
                run += 1
            longest_stale = max(longest_stale, run)
        report.append({
            "Symbol": sym,
            "NonNullDays": int(non_null),
            "PctMissing": round(100*pct_missing,2),
            "Zeros": int(zeros),
            "Negatives": int(negatives),
            "UniqueValues": int(unique_vals),
            "FirstValidDate": str(first_valid) if first_valid is not None else "",
            "LastValidDate": str(last_valid) if last_valid is not None else "",
            "MinPrice": float(min_p) if not pd.isna(min_p) else np.nan,
            "MaxPrice": float(max_p) if not pd.isna(max_p) else np.nan,
            "MaxAbsDailyReturn": max_abs,
            "Days>50%Move": days_over_50,
            "LongestStaleRun": int(longest_stale)
        })
    report_df = pd.DataFrame(report).sort_values(["PctMissing","NonNullDays"], ascending=[False,True])
    # filter clean symbols
    clean_symbols = report_df[report_df["NonNullDays"] >= min_history_days]["Symbol"].tolist()
    # also drop symbols with > pct_missing_threshold missing:
    clean_symbols = [s for s in clean_symbols if report_df[report_df["Symbol"]==s]["PctMissing"].iloc[0] <= 100*pct_missing_threshold]
    # NEW: Remove stocks with Days>50%Move > 0 from clean_symbols
    symbols_to_remove_due_to_spikes = report_df[report_df["Days>50%Move"] > 0]["Symbol"].tolist()
    clean_symbols = [s for s in clean_symbols if s not in symbols_to_remove_due_to_spikes]

    clean_price_df = price_series[clean_symbols].copy()
    # Save outputs
    out_file = Path(output_clean_path)
    with pd.ExcelWriter(out_file, engine="openpyxl", datetime_format="YYYY-MM-DD") as writer:
        price_series.reset_index().to_excel(writer, sheet_name="RawPriceMatrix", index=False)
        clean_price_df.reset_index().to_excel(writer, sheet_name="CleanPriceMatrix", index=False)
        report_df.to_excel(writer, sheet_name="Symbol_Summary", index=False)
        report_df[report_df["PctMissing"]>100*pct_missing_threshold].to_excel(writer, sheet_name="High_Missing", index=False)
        report_df[report_df["Days>50%Move"]>0].to_excel(writer, sheet_name="Extreme_Spikes", index=False)
    print("Saved cleaned price matrix and summary to:", out_file)
    return out_file, clean_price_df, report_df

# -------------------------
# Module 3: Momentum engines
# -------------------------
def get_rebalance_dates(price_index, freq='fortnightly'):
    """price_index is a DatetimeIndex sorted ascending (daily trading days).
    freq: 'weekly' (every Friday), 'fortnightly' (every other Friday), 'monthly' (last trading day of month).
    Returns list of pd.Timestamp objects for rebalance days (subset of index).
    """
    dates = pd.Series(sorted(price_index.unique()))
    if freq == 'monthly':
        rb = dates.groupby(dates.dt.to_period('M')).max().tolist()
    elif freq == 'weekly':
        fridays = dates[dates.dt.weekday == 4].tolist()
        rb = fridays
    elif freq == 'fortnightly':
        fridays = dates[dates.dt.weekday == 4].tolist()
        rb = fridays[::2] if len(fridays) >= 2 else dates[13::14].tolist()
    else:
        raise ValueError("freq must be one of weekly/fortnightly/monthly")
    return rb

def compute_scores_and_signals(price_df, rebalance_dates):
    """
    price_df: DataFrame indexed by Date with symbols as columns (Adj-close adjusted).
    rebalance_dates: list of dates to compute signals on.
    Returns signals_df with columns: RebalanceDate, Symbol, Price, R52_ex1, R26, R13, R4, Score, DMA50, DMA200
    """
    signals = []
    symbols = price_df.columns.tolist()
    # convert to numpy arrays per symbol to speed calculations
    for sym in symbols:
        ser = pd.to_numeric(price_df[sym], errors='coerce').dropna()
        if ser.shape[0] < 20:  # insufficient
            continue
        dates_idx = ser.index.values.astype('datetime64[ns]')
        prices_arr = ser.values.astype(float)
        # precompute DMAs using pandas on ser
        s_pd = ser
        dma50 = s_pd.rolling(window=50, min_periods=10).mean()
        dma200 = s_pd.rolling(window=200, min_periods=30).mean()
        for dt in rebalance_dates:
            if dt < s_pd.index.min() or dt > s_pd.index.max():
                continue
            # helper to compute cumulative return by N observations
            def cumret_by_n(n, exclude_last=0):
                pos = np.searchsorted(dates_idx, np.datetime64(dt), side='right') - 1
                if pos - exclude_last - n < 0 or pos < 0:
                    return np.nan
                start = pos - exclude_last - n
                start_price = prices_arr[start]
                end_price = prices_arr[pos - exclude_last]
                if start_price == 0 or np.isnan(start_price) or np.isnan(end_price): return np.nan
                return end_price / start_price - 1
            R52 = cumret_by_n(260, exclude_last=5)
            R26 = cumret_by_n(130, exclude_last=0)
            R13 = cumret_by_n(65, exclude_last=0)
            R4  = cumret_by_n(20, exclude_last=0)
            pos = np.searchsorted(dates_idx, np.datetime64(dt), side='right') - 1
            price = float(prices_arr[pos]) if pos>=0 and not np.isnan(prices_arr[pos]) else np.nan
            # DMA values at pos (get via dma series)
            dma50_v = float(dma50.iloc[pos]) if pos>=0 and not pd.isna(dma50.iloc[pos]) else np.nan
            dma200_v = float(dma200.iloc[pos]) if pos>=0 and not pd.isna(dma200.iloc[pos]) else np.nan
            # score
            if all(pd.isna([R52,R26,R13,R4])):
                score = np.nan
            else:
                score = 0.5*(R52 if not pd.isna(R52) else 0) + 0.25*(R26 if not pd.isna(R26) else 0) + 0.15*(R13 if not pd.isna(R13) else 0) + 0.10*(R4 if not pd.isna(R4) else 0)
            signals.append({'RebalanceDate': pd.Timestamp(dt),'Symbol': sym, 'Price': price,
                            'DMA50': dma50_v, 'DMA200': dma200_v, 'R52_ex1': R52, 'R26': R26, 'R13': R13, 'R4': R4, 'Score': score})
    signals_df = pd.DataFrame(signals)
    signals_df = signals_df.dropna(subset=['Score']).sort_values(['RebalanceDate','Score'], ascending=[True, False]).reset_index(drop=True)

    # Drop rows where DMA50 or DMA200 are NaN, as these cannot be used for eligibility
    signals_df = signals_df.dropna(subset=['DMA50', 'DMA200'])

    # apply DMA eligibility flag / Conciously DMA 50 has been multiplied by .9.
    # So that immediately if stock crosses below DMA50 it is not rejected.
    signals_df['Eligible'] = (signals_df['Price'] > 0.9*signals_df['DMA50']) & (signals_df['Price'] > signals_df['DMA200']) & (signals_df['DMA50'] > signals_df['DMA200'])
    return signals_df

# -------------------------
# Module 4: Backtest and reporting
# -------------------------
def backtest_grid_and_report(price_df, signals_df, rebalance_dates, freq_label, topN_list=(5,10,15,20), excludeK_list=(0,1,2,3), weighting='equal', out_xlsx=OUTPUT_DIR/"results_fortnightly.xlsx"):
    """
    Run grid backtests on signals_df and price_df for given rebalance_dates,
    compute metrics, and produce an Excel workbook with:
      - Results grid
      - Top strategies (top 10 by Sharpe)
      - For each selected top strategy: holdings per rebalance (Symbol, Weight, Price_in, Price_out, Return)
    """
    print("Running backtest grid for", freq_label)
    dates_sorted = sorted(rebalance_dates)
    next_map = {dates_sorted[i]: (dates_sorted[i+1] if i < len(dates_sorted)-1 else None) for i in range(len(dates_sorted))}
    rows = []
    holdings_store = {}  # key (topN,excl) -> DataFrame holdings
    periods_store = {}
    for topN in topN_list:
        for excl in excludeK_list:
            port_rets = []
            per_period_holdings = []
            for d in dates_sorted:
                nd = next_map[d]
                if nd is None: continue

                # Get all signals for the current rebalance date, including non-eligible ones.
                # Only drop NaNs for 'Score' as these are truly unrankable.
                df_d = signals_df[signals_df['RebalanceDate']==d].dropna(subset=['Score']).copy()

                if df_d.empty:
                    port_rets.append(np.nan); continue

                # Create a combined sorting key: prioritize Eligible=True, then by Score (descending).
                # A large multiplier (e.g., 1000) ensures eligible stocks are always ranked higher than non-eligible ones
                # if their scores are in a reasonable range (e.g., -1 to 1).
                df_d['SortKey'] = df_d['Eligible'].astype(int) * 1000 + df_d['Score']

                # Sort all stocks by this new SortKey (descending).
                df_d_sorted = df_d.sort_values(by='SortKey', ascending=False).reset_index(drop=True)

                # Apply exclusion first: remove the top 'excl' stocks from the prioritized list.
                df_after_excl = df_d_sorted.iloc[excl:].copy()

                # Now, from the remaining stocks, select the top 'topN'.
                # This ensures we pick 'topN' stocks if `df_after_excl` has at least 'topN' rows.
                # If `df_after_excl` has fewer than 'topN' rows, it will select all available remaining stocks.
                sel = df_after_excl.head(topN).copy()

                if sel.empty:
                    port_rets.append(np.nan); continue

                syms = sel['Symbol'].values
                # get prices at d and nd
                try:
                    p_now = price_df.loc[price_df.index <= d].iloc[-1].reindex(syms).values.astype(float)
                    p_next = price_df.loc[price_df.index <= nd].iloc[-1].reindex(syms).values.astype(float)
                except Exception:
                    port_rets.append(np.nan); continue
                with np.errstate(divide='ignore', invalid='ignore'):
                    rets = (p_next / p_now) - 1.0
                weights = np.repeat(1/len(syms), len(syms))
                valid = ~np.isnan(rets)
                if not np.any(valid):
                    port_rets.append(np.nan); continue
                w = weights[valid]; w = w / np.sum(w)
                port_ret = float(np.nansum(w * rets[valid]))
                port_rets.append(port_ret)
                # store holdings rows
                for i, sym in enumerate(syms):
                    per_period_holdings.append({'RebalanceDate': d, 'NextDate': nd, 'Symbol': sym, 'Rank': int(sel.iloc[i]['Rank']) if 'Rank' in sel.columns else np.nan, 'Price_in': float(p_now[i]) if not np.isnan(p_now[i]) else np.nan, 'Price_out': float(p_next[i]) if not np.isnan(p_next[i]) else np.nan, 'Return': float(rets[i]) if not np.isnan(rets[i]) else np.nan, 'Weight': float(weights[i])})
            s = pd.Series(port_rets).dropna()
            if s.empty:
                metrics = {'mean_period':np.nan,'annual_return':np.nan,'annual_vol':np.nan,'sharpe':np.nan,'mdd':np.nan,'obs':0}
            else:
                meanp = s.mean(); stdp = s.std(ddof=1)
                per_year = 12 if freq_label=='monthly' else (26 if freq_label=='fortnightly' else 52)
                ann = (1+meanp)**per_year - 1
                annvol = stdp * math.sqrt(per_year)
                sharpe = ann / annvol if (not np.isnan(annvol) and annvol>0) else np.nan
                cum = (1+s).cumprod(); peak = cum.cummax(); dd = (cum-peak)/peak; mdd = dd.min()
                metrics = {'mean_period':meanp,'annual_return':ann,'annual_vol':annvol,'sharpe':sharpe,'mdd':mdd,'obs':len(s)}
            rows.append({'freq':freq_label,'topN':topN,'excludeK':excl,'weighting':weighting, **metrics})
            key = (topN,excl)
            if len(per_period_holdings)>0:
                holdings_store[key] = pd.DataFrame(per_period_holdings)
                periods_store[key] = pd.DataFrame({'RebalanceDate':sorted(set([r['RebalanceDate'] for r in per_period_holdings])), 'PortfolioReturn': np.nan})
                # we'll compute per-period returns separately above, but for convenience we retain holdings
    results_df = pd.DataFrame(rows)
    # save results
    out = Path(out_xlsx)
    with pd.ExcelWriter(out, engine="openpyxl", datetime_format="YYYY-MM-DD") as writer:
        results_df.to_excel(writer, sheet_name="ResultsGrid", index=False)
        # top strategies by sharpe
        top10 = results_df.sort_values(['sharpe','annual_return'], ascending=[False,False]).head(10)
        top10.to_excel(writer, sheet_name="Top10_BySharpe", index=False)
        # also write holdings for each top10
        for idx, row in top10.iterrows():
            key = (int(row['topN']), int(row['excludeK']))
            df_hold = holdings_store.get(key)
            if df_hold is None:
                pd.DataFrame([{"Note":"No holdings for this configuration"}]).to_excel(writer, sheet_name=f"H_{row['topN']}_E{row['excludeK']}"[:31], index=False)
            else:
                df_hold.to_excel(writer, sheet_name=f"H_{row['topN']}_E{row['excludeK']}"[:31], index=False)
    print("Saved backtest workbook to:", out)
    return out, results_df

# -------------------------
# Orchestration / UI
# -------------------------
def main():
    print("Momentum pipeline interactive runner.")
    print("Default ticker list path:", DEFAULT_INPUT_TICKER_XLSX)
    print("Outputs are written into:", OUTPUT_DIR.resolve())
    while True:
        print("\nMenu:")
        print("1) Module 1 — Download data from Yahoo (reads tickers from an Excel file)")
        print("2) Module 2 — Clean data (build price matrix & symbol report)")
        print("3) Module 3 — Run Momentum (weekly/fortnightly/monthly) -> compute signals")
        print("4) Module 4 — Backtest grid & Top10 report (requires cleaned price matrix + signals)")
        print("5) Exit")
        choice = input("Choose option (1-5): ").strip()
        if choice=="1":
            inpath = input(f"Ticker list path (press Enter for default {DEFAULT_INPUT_TICKER_XLSX}): ").strip() or DEFAULT_INPUT_TICKER_XLSX
            out = input("Output XLSX path (press Enter for default './output/yahoo_data.xlsx'): ").strip() or "./output/yahoo_data.xlsx"
            years_str = input("Number of years to download (press Enter for default 2 years): ").strip()
            years = int(years_str) if years_str.isdigit() else 2
            print(f"About to download tickers from {inpath} for {years} years and save to {out}")
            if not confirm("Confirm run downloader? (y/n): "): continue
            download_yahoo_data(inpath, out, years=years)
        elif choice=="2":
            yahoo_path = input("Path to yahoo data XLSX (press Enter for './output/yahoo_data.xlsx'): ").strip() or "./output/yahoo_data.xlsx"
            if not Path(yahoo_path).exists():
                print("File not found:", yahoo_path); continue
            out_clean = input("Clean output path (press Enter for './output/cleaned_price_matrix.xlsx'): ").strip() or "./output/cleaned_price_matrix.xlsx"
            print("This will build a price matrix and symbol report then save to", out_clean)
            if not confirm("Confirm data cleanup? (y/n): "): continue
            clean_data(yahoo_path, out_clean)
        elif choice=="3":
            clean_path = input("Path to cleaned price matrix XLSX (press Enter for './output/cleaned_price_matrix.xlsx'): ").strip() or "./output/cleaned_price_matrix.xlsx"
            if not Path(clean_path).exists():
                print("Clean file not found:", clean_path); continue

            # Original: sheet_choice = input("Which frequency? Enter weekly / fortnightly / monthly: ").strip().lower()
            freq_input = input("Which frequency? Enter 1 for weekly, 2 for fortnightly, 3 for monthly: ").strip()
            if freq_input == '1':
                sheet_choice = 'weekly'
            elif freq_input == '2':
                sheet_choice = 'fortnightly'
            elif freq_input == '3':
                sheet_choice = 'monthly'
            else:
                print("Invalid choice. Please enter 1, 2, or 3."); continue

            print(f"About to compute signals for {sheet_choice}.")
            if not confirm("Confirm compute signals? (y/n): "): continue
            # load clean matrix
            xl = pd.ExcelFile(clean_path)
            if "CleanPriceMatrix" in xl.sheet_names:
                price_df = pd.read_excel(clean_path, sheet_name="CleanPriceMatrix", parse_dates=["Date"]).set_index("Date").sort_index()
            else:
                price_df = pd.read_excel(clean_path, sheet_name=xl.sheet_names[0], parse_dates=["Date"]).set_index("Date").sort_index()
            rebalance_dates = get_rebalance_dates(price_df.index, freq=sheet_choice)
            print("Rebalance dates count:", len(rebalance_dates))
            signals_df = compute_scores_and_signals(price_df, rebalance_dates)
            sig_out = Path(OUTPUT_DIR)/f"signals_{sheet_choice}.csv"
            signals_df.to_csv(sig_out, index=False)
            print("Signals saved to:", sig_out)
        elif choice=="4":
            price_path = input("Path to clean price matrix XLSX (Enter default './output/cleaned_price_matrix.xlsx'): ").strip() or "./output/cleaned_price_matrix.xlsx"

            # New frequency input for Module 4
            freq_input_m4 = input("Enter frequency label used for signals (1 for weekly, 2 for fortnightly, 3 for monthly): ").strip()
            if freq_input_m4 == '1':
                freq_label = 'weekly'
            elif freq_input_m4 == '2':
                freq_label = 'fortnightly'
            elif freq_input_m4 == '3':
                freq_label = 'monthly'
            else:
                print("Invalid frequency choice. Please enter 1, 2, or 3."); continue

            # Updated signals_csv to use the new freq_label
            signals_csv = input(f"Path to signals CSV (press Enter for default './output/signals_{freq_label}.csv'): ").strip() or f"./output/signals_{freq_label}.csv"

            # Original: freq_label = input("Enter frequency label used for signals (weekly/fortnightly/monthly): ").strip().lower()

            if not Path(price_path).exists():
                print("Price file not found:", price_path); continue
            if not Path(signals_csv).exists():
                print("Signals file not found:", signals_csv); continue
            if freq_label not in ("weekly","fortnightly","monthly"):
                print("Invalid freq"); continue # This check is redundant with the new input logic but harmless
            if not confirm("Run backtest and report? (y/n): "): continue
            price_df = pd.read_excel(price_path, sheet_name="CleanPriceMatrix", parse_dates=["Date"]).set_index("Date").sort_index()
            signals_df = pd.read_csv(signals_csv, parse_dates=["RebalanceDate"])
            out_xlsx = Path(OUTPUT_DIR)/f"results_{freq_label}.xlsx"
            backtest_grid_and_report(price_df, signals_df, sorted(signals_df['RebalanceDate'].unique()), freq_label, out_xlsx=out_xlsx)
        elif choice=="5":
            print("Exiting.")
            break
        else:
            print("Unknown choice. Try again.")

if __name__ == "__main__":
    main()